# Setup

In [ ]:
# Check out the agilerobotics branch of some of our packages:
Pkg.checkout("RigidBodyDynamics", "agilerobotics")

if !isdir(Pkg.dir("DrakeVisualizer"))
    Pkg.clone("https://github.com/rdeits/DrakeVisualizer.jl")
    Pkg.checkout("DrakeVisualizer", "robocon2017")
    Pkg.build("DrakeVisualizer")
end

Pkg.add("RigidBodyTreeInspector")
Pkg.checkout("RigidBodyTreeInspector", "robocon2017")

In [ ]:
using RigidBodyDynamics
using StaticArrays
using Plots
using RigidBodyTreeInspector
using DrakeVisualizer

# Creating, simulating, and visualizing a double pendulum `Mechanism`

We're going to create a simple `Mechanism` that represents a [double pendulum](https://en.wikipedia.org/wiki/Double_pendulum).
One way of creating a `Mechanism` is by parsing a [URDF](http://wiki.ros.org/urdf) file:

In [ ]:
urdf = "data/doublependulum.urdf"
doublependulum = parse_urdf(Float64, urdf)

A `Mechanism` stores the joint/rigid body layout, but no state information. State information is separated out into a `MechanismState` object:

In [ ]:
const state = MechanismState(Float64, doublependulum)

Next, we'll open a DrakeVisualizer window and start visualizing the double pendulum.

In [ ]:
DrakeVisualizer.any_open_windows() || DrakeVisualizer.new_window();
vis = Visualizer(parse_urdf(urdf, doublependulum));

Let's first set the configurations and velocities of the joints:

In [ ]:
RigidBodyTreeInspector.inspect!(state, vis);

Basic simulation is easy:

In [ ]:
ts, qs, vs = simulate(state, 5., Δt = 1e-3);

After which we can animate the results:

In [ ]:
RigidBodyTreeInspector.animate(vis, doublependulum, ts, qs)

Or plot them using e.g. Plots.jl:

In [ ]:
plot(ts, collect(q[1] for q in qs), xlabel = "Time [s]", ylabel = "Angle [rad]", lab = "Shoulder")

# The basic design of  `Mechanism` and `MechanismState`

A `Mechanism` is an interconnection of rigid bodies and joints, without any state information:

In [ ]:
collect(bodies(doublependulum))

In [ ]:
collect(joints(doublependulum))

A `MechanismState` has all the state information, e.g. the configurations and velocities of all the joints:

In [ ]:
configuration(state)

In [ ]:
velocity(state)

It also has various cached computation results meant to reduce double work while computing various kinematic/dynamic quantities.

# Kinematics

Let's unpack the list of joints into separate variables:

In [ ]:
fixedjoint, shoulder, elbow = (joints(doublependulum)...)

Many objects have frame annotations in the form of `CartesianFrame3D` objects:

In [ ]:
shoulder.frameAfter

In [ ]:
shoulder.frameBefore

Here's how you create a point in the frame after the elbow joint:

In [ ]:
p = Point3D(elbow.frameAfter, SVector(1., 2., 3.))

Transform it to world frame in the current state:

In [ ]:
p = transform(state, p, root_frame(doublependulum))

Now suppose we have a displacement we defined in `elbow.frameAfter`:

In [ ]:
displacement = FreeVector3D(elbow.frameAfter, SVector(2., 3., 4.))

and we want to add it to `p`:

In [ ]:
p + displacement

Excercise: how to fix?

You can also ask for the homogeneous transform to world:

In [ ]:
transform_to_root(state, elbow.frameAfter)

Or a relative transform:

In [ ]:
relative_transform(state, elbow.frameAfter, shoulder.frameAfter)

Here's the center of mass of the double pendulum in the given state:

In [ ]:
center_of_mass(state)

Motion between bodies is represented using `Twist`s:

In [ ]:
twist = relative_twist(state, elbow.frameAfter, shoulder.frameBefore)

In [ ]:
transform(state, twist, elbow.frameAfter)

# Dynamics

A `MechanismState` can also be used to compute quantities related to the dynamics of the `Mechanism`. Here we compute the mass matrix:

In [ ]:
mass_matrix(state)

Here's the documentation for `mass_matrix` by the way:

In [ ]:
?mass_matrix

Note that there is also a zero-allocation version, `mass_matrix!` (the `!` at the end of a method is a Julia convention signifying that the function is 'in-place', i.e. modifies its input data).

We can do inverse dynamics as follows (note again that there is a non-allocating version of this method as well):

In [ ]:
v̇ = [2.; 3.] # the joint acceleration vector, i.e., the time derivative of the joint velocity vector v
inverse_dynamics(state, v̇)

For forward dynamics, one first creates a `DynamicsResult` object, which preallocates a bunch of stuff:

In [ ]:
result = DynamicsResult(Float64, doublependulum);

In [ ]:
dynamics!(result, state)

In [ ]:
result.v̇

# Symbolic dynamics

## Setup

In [ ]:
# Make SymPy use version of Python downloaded through Julia's package system instead of system Python
ENV["PYTHON"]=""
Pkg.checkout("SymPy")
Pkg.build("SymPy")

In [ ]:
using SymPy

In [ ]:
doublependulumsym = parse_urdf(Sym, urdf)
remove_fixed_joints!(doublependulumsym)

## Create symbolic parameters
* Masses: $m_1, m_2$
* Mass moments of inertia (about center of mass): $I_1, I_2$
* Link lengths: $l_1, l_2$
* Center of mass locations (w.r.t. preceding joint axis): $c_1, c_2$
* Gravitational acceleration: $g$

In [ ]:
inertias = @syms m_1 m_2 I_1 I_2 positive = true
lengths = @syms l_1 l_2 c_1 c_2 real = true
gravitationalAcceleration = @syms g real = true
[inertias..., lengths..., gravitationalAcceleration...]'

Unpack bodies and joints:

In [ ]:
world, upperlink, lowerlink = (bodies(doublependulumsym)...)
shoulder, elbow = (joints(doublependulumsym)...);

Set symbolic parameters:

In [ ]:
axis = shoulder.jointType.axis
spatial_inertia!(upperlink, SpatialInertia(shoulder.frameAfter, I_1 * axis * axis', SVector(0, 0, -c_1), m_1))
spatial_inertia!(lowerlink, SpatialInertia(elbow.frameAfter, I_2 * axis * axis', SVector(0, 0, -c_2), m_2))
add_frame!(upperlink, Transform3D(elbow.frameBefore, shoulder.frameAfter, SVector(0, 0, l_1)));

Create a symbolic `MechanismState`:

In [ ]:
statesym = MechanismState(Sym, doublependulumsym);

Set the joint configuration and joint velocity vectors of the `MechanismState` to new vectors of symbolic variables:

In [ ]:
configuration(statesym)[:] = [symbols("q_$i", real = true) for i = 1 : num_positions(statesym)]

In [ ]:
velocity(statesym)[:] = [symbols("v_$i", real = true) for i = 1 : num_positions(statesym)]

Compute the mass matrix in symbolic form:

In [ ]:
Msym = mass_matrix(statesym)
map!(simplify, Msym.data) # Note: M is a Symmetric matrix type; need to simplify the underlying data
Msym

Kinetic energy:

In [ ]:
simplify(kinetic_energy(statesym))

# Modifying `Mechanism`s

`Mechanism`s can be modified in various ways. Here's our original double pendulum:

In [ ]:
doublependulum

Removing fixed joints:

In [ ]:
nofixedjoints = deepcopy(doublependulum)
remove_fixed_joints!(nofixedjoints)

Converting to maximal coordinates:

In [ ]:
dp_maxcoord, _ = maximal_coordinates(nofixedjoints)
dp_maxcoord

In [ ]:
collect(joints(dp_maxcoord)) # really just the `tree' joints

In [ ]:
collect(edge.joint for edge in dp_maxcoord.nonTreeEdges) # the original joints become non-tree joints

In [ ]:
num_positions(dp_maxcoord)

In [ ]:
num_velocities(dp_maxcoord)

Creating a new mechanism from a subtree:

In [ ]:
upperlink = collect(bodies(doublependulum))[3]
submechanism(doublependulum, upperlink)

Also available: attaching one `Mechanism` to another, rerooting.